In [23]:
import tensorflow as tf
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import konlpy

# 데이터 불러오기

In [2]:
data = pd.read_csv('data_fin_.csv',encoding = 'cp949')
data.head()

,userid,courseid,course,rating,강원도,경상북도,경상남도,전라북도,전라남도,충청도,...,나무,유적지,맛집여행,호캉스,감성카페,애견동반ㅇ,봄,여름,가을,겨울
0,1,1,"유진게찜,엘꼬시네로(스페인음식)",10,1,0,0,0,0,0,...,0,0,1,0,0.0,0,1,1,1,1
1,1,2,"동명정류장카페,바다를본돼지",10,0,0,0,0,0,0,...,0,0,1,0,1.0,0,1,1,1,1
2,1,3,"몽상드애월카페,아기해녀의집",10,0,0,0,0,0,0,...,0,0,1,0,1.0,0,1,1,1,1
3,1,4,대전운주계곡,8,0,0,0,0,0,0,...,1,0,0,0,0.0,0,1,1,1,1
4,5,5,"양양쏠비치송이(한식),호텔내미술관",10,1,0,0,0,0,0,...,0,0,0,0,0.0,0,1,1,1,1


In [3]:
data = data[['userid','courseid','course','rating']]
data.head()

,userid,courseid,course,rating
0,1,1,"유진게찜,엘꼬시네로(스페인음식)",10
1,1,2,"동명정류장카페,바다를본돼지",10
2,1,3,"몽상드애월카페,아기해녀의집",10
3,1,4,대전운주계곡,8
4,5,5,"양양쏠비치송이(한식),호텔내미술관",10


In [6]:
data['viewed'] = np.nan #rating이 7 이상인 것들은 다시 갈 곳으로 간주, 그러지 않으면 다시 가지 않을 곳으로 간주
for a in range(len(data)):
    if(data['rating'][a] >= 7):
        data['viewed'][a] = 1
    else:
        data['viewed'][a] = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
data.head()

,userid,courseid,course,rating,viewed
0,1,1,"유진게찜,엘꼬시네로(스페인음식)",10,1.0
1,1,2,"동명정류장카페,바다를본돼지",10,1.0
2,1,3,"몽상드애월카페,아기해녀의집",10,1.0
3,1,4,대전운주계곡,8,1.0
4,5,5,"양양쏠비치송이(한식),호텔내미술관",10,1.0


In [22]:
X_data = data['course']
y_data = data['viewed']
print('course 개수: {}'.format(len(X_data)))
print('viewed 개수: {}'.format(len(y_data)))

course 개수: 512
viewed 개수: 512


# 토큰화

In [24]:
normalized_text = []
for string in X_data.tolist():
    try:
        tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ", string.lower())
    except Exception as e:
        print(string)
        break
    normalized_text.append(tokens)
print(normalized_text[:5])

['유진게찜 엘꼬시네로 스페인음식 ', '동명정류장카페 바다를본돼지', '몽상드애월카페 아기해녀의집', '대전운주계곡', '양양쏠비치송이 한식 호텔내미술관']


In [32]:
data.course = normalized_text
for sentence in data['course']:
    print(sentence)

유진게찜 엘꼬시네로 스페인음식 
동명정류장카페 바다를본돼지
몽상드애월카페 아기해녀의집
대전운주계곡
양양쏠비치송이 한식 호텔내미술관
가평선어치고개집
양양파마씨 파마씨펍
해운대다솥 한식 카페루프탑 로컬 펍 라라랜드
청평팀어거스터수상레저
부산본전돼지국밥 초랑밀면 신발원
인천차이나타운유가복 흑백사진관
청도빠삐용캠프 수제버거 와인터널
청도읍성 꽃자리 한옥카페 
대구중앙떡볶이 오차야미테
대구엘아르카레푸스진천점 단골식당 석쇠불고기 김광석길 루프탑카페편수성
전주에루화본점 다기양조장
전주아원고택 삼백집본점 한옥마을
당진아이미술관 왜목마을해수욕장 빙빙반점
울산주전몽돌해변 당사해양낚시공원 태화강국가정원 십대리숲
안성금광호수공원 까페호미
대구김광석길 그린곱창막창
양평빠빠에피카페 사나산아 한식 
양평두물머리연핫도그 오르다온
서귀포협재해변 새별오름 함덕카페 
양평용문산관광단지 양평더그림카페 아우름의식탁
단양한옥마을 단양카페산 도담삼봉 만천하스카이워크
해운대이기대해안산책로 풍원장
서귀포어애랑 한림공원 앤트러사이트 성이시돌목장 송악산둘레길 앙끄레국수 황우지해안
서귀포카페바다다 용눈이오름 어멍이해녀 세화해변 큰엉해안경승지 애월
남해다랭이마을 상주은모래비치 독일마을 양모리학교
거제바람의언덕 거제몽돌해변
양양멍비치 사랑대게
송도솔찬공원 반메이 센트럴파크
강릉강문해변 대관령순수양떼목장
강릉남애항스카이워크 둔내역한우
강릉안목해변 강릉짬뽕순두부동화가든본점 경포해변 카페툇마루
강릉허균허난설헌기념관 강문해변
춘천소양강스카이워크 춘천시풍물시장 대원당 춘천의암호스카이워크
속초아바이마을 바다정원 통일전망대
춘천이상원미술관 김유정폐역
속초대포항튀김골목 씨씨카페
강릉소돌아들바위공원 주문진항 도깨비촬영지 보헤미안박이추커피 사천포구 강릉커피콩빵 강릉카페거리 하슬리아트월드 정동진해변 정동심곡바다부채길
강릉하슬라아트월드
인천월미바다열차 차이나타운 송월동동화마을 월미도 북성포구
포천전통술박물관산사원
인천무의도 실미도해수욕장 하나개해수욕장
광명동굴 삼백돈돈가츠
양평두물머리 나인블럭 테레로사 대복식당불고기가게
가평스위스테마

In [44]:
data_1 = data[data['userid'] ==  1]
data_2 = data[data['userid'] ==  2]
data_3 = data[data['userid'] ==  3]
data_4 = data[data['userid'] ==  4]
data_5 = data[data['userid'] ==  5]
data_6 = data[data['userid'] ==  6]
data_7 = data[data['userid'] ==  7]

In [33]:
from konlpy.tag import Okt  
okt = Okt()

In [34]:
X_token=[]
for sentence in data_1['course']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    #temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_token.append(temp_X)

In [35]:
print(X_token[:3])

[['유진', '게', '찜', '엘꼬시네', '로', '스페인', '음식'], ['동명', '정류장', '카페', '바다', '를', '본', '돼지'], ['몽상', '드애월', '카페', '아기', '해녀', '의', '집']]


# 정수 인코딩 수행

In [36]:
max_words = 35000
tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
tokenizer.fit_on_texts(X_token) 
X_token = tokenizer.texts_to_sequences(X_token)

In [37]:
print(X_token[:3])

[[709, 285, 710, 711, 184, 462, 463], [464, 712, 1, 64, 465, 256, 144], [713, 714, 1, 715, 286, 93, 42]]


In [38]:
word_to_index = tokenizer.word_index

# 전체 데이터 길이 분포 확인

In [39]:
vocab_size = len(word_to_index) + 1
print('단어 집합의 크기: {}'.format((vocab_size)))

단어 집합의 크기: 1568


In [40]:
print('train 코스의 최대 길이 :',max(len(l) for l in X_token))
print('평균 길이 :',sum(map(len, X_token))/len(X_token))

train 코스의 최대 길이 : 36
평균 길이 : 10.310546875


In [41]:
max_len = 36
X_data = pad_sequences(X_token, maxlen=max_len)
print("data shape: ", X_data.shape)

data shape:  (512, 36)


In [42]:
y_data = np.array(y_data).reshape(-1, 1)

In [43]:
print(X_data[:3])
print(y_data[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0 709 285 710 711 184 462 463]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0 464 712   1  64 465 256 144]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0 713 714   1 715 286  93  42]]
[[1.]
 [1.]
 [1.]]


# 데이터 분리

In [45]:
data_1.head()

,userid,courseid,course,rating,viewed
0,1,1,유진게찜 엘꼬시네로 스페인음식,10,1.0
1,1,2,동명정류장카페 바다를본돼지,10,1.0
2,1,3,몽상드애월카페 아기해녀의집,10,1.0
3,1,4,대전운주계곡,8,1.0
168,1,169,잠실롯데시그니엘호텔 롯데월드,10,1.0


In [21]:
cdata_1 = data_1['course']
cdata_2 = data_2['course']
cdata_3 = data_3['course']
cdata_4 = data_4['course']
cdata_5 = data_5['course']
cdata_6 = data_6['course']
cdata_7 = data_7['course']
print('course 개수: {}'.format(len(cdata_1)))
print('course 개수: {}'.format(len(cdata_2)))
print('course 개수: {}'.format(len(cdata_3)))
print('course 개수: {}'.format(len(cdata_4)))
print('course 개수: {}'.format(len(cdata_5)))
print('course 개수: {}'.format(len(cdata_6)))
print('course 개수: {}'.format(len(cdata_7)))

course 개수: 70
course 개수: 43
course 개수: 182
course 개수: 88
course 개수: 82
course 개수: 3
course 개수: 44
